In [ ]:
#上証指數
import akshare as ak

#shenzhen

stock_szse_summary_df = ak.stock_szse_summary(date="20250221")
print(stock_szse_summary_df)


#shanghai 
stock_sse_summary_df = ak.stock_szse_summary(date="20250221")
print(stock_sse_summary_df)

In [ ]:
import akshare as ak
import pandas as pd
# 获取东方财富网的概念板块数据

industry_board_df = ak.stock_board_industry_name_em()

industry_board_df.to_csv(r"C:\Quant\quant_HC\data\stocks\industry_board_df.csv", index=False)
print(industry_board_df)


In [ ]:
import akshare as ak

# 检查 akshare 版本（需 >= 1.10.0）
print("akshare 版本:", ak.__version__)

# 获取概念板块数据
try:
    df = ak.stock_board_concept_name_em()
    print("前5行数据示例:\n", df.head())
    print("\n列名检查:", df.columns.tolist())
    
    # 检查关键字段是否有有效数据
    if df["涨跌幅"].sum() == 0:
        print("警告：涨跌幅全为0，可能未在交易时段运行！")
except Exception as e:
    print("接口调用失败，错误信息:", e)

In [ ]:
import akshare as ak

stock_zh_a_spot_em_df = ak.stock_zh_a_spot_em()
print(stock_zh_a_spot_em_df)


In [ ]:
import akshare as ak

# Fetch the stock data
stock_zh_a_spot_em_df = ak.stock_zh_a_spot_em()

# Save the DataFrame to a CSV file (corrected file path)
stock_zh_a_spot_em_df.to_csv(r"C:\Quant\quant_HC\data\stocks\stock_zh_a_spot_em.csv", index=False)

print("CSV file has been saved as 'stock_zh_a_spot_em.csv' in the specified directory.")


In [ ]:
# 综合筛选大资金流入候选股
candidates = stock_zh_a_spot_em_df[
    (stock_zh_a_spot_em_df["量比"] > 2) &
    (stock_zh_a_spot_em_df["换手率"] > 15) &
    (stock_zh_a_spot_em_df["涨跌幅"] > 5) &
    (stock_zh_a_spot_em_df["成交额"] > 5e8)
]

print("大资金流入候选股：")
print(candidates[["代码", "名称", "最新价", "涨跌幅", "换手率", "量比", "成交额"]])

In [ ]:
# 综合筛选大资金流入候选股
candidates_2 = stock_zh_a_spot_em_df[
    (stock_zh_a_spot_em_df["涨跌幅"] > 5)
]

print("大资金流入候选股：")
print(candidates_2[["代码", "名称", "最新价", "涨跌幅", "换手率", "量比", "成交额"]])

In [ ]:
import pandas as pd

# Load the CSV file into a DataFrame
csv_file_path = r"C:\Quant\quant_HC\data\stocks\stock_zh_a_spot_em.csv"
stock_df = pd.read_csv(csv_file_path)

# Display the first few rows of the DataFrame
# 综合筛选大资金流入候选股
candidates_1 = stock_df[
    (stock_df["量比"] > 2) &
    (stock_df["换手率"] > 15) &
    (stock_df["成交额"] > 5e8)
]

print("大资金流入候选股：")
print(candidates_1[["代码", "名称", "最新价", "涨跌幅", "换手率", "量比", "成交额"]])

In [ ]:
import pandas as pd

# 将 '代码' 列转换为字符串类型
candidates_1["代码"] = candidates_1["代码"].astype(str)
candidates_2["代码"] = candidates["代码"].astype(str)

# 通过 '代码' 列进行合并
common_df = pd.merge(candidates_1, candidates, on="代码", how="inner")

# 打印重合的行
print("重合的股票：")
print(common_df[["代码", "名称_x", "最新价_x", "涨跌幅_x", "换手率_x", "量比_x", "成交额_x"]])

# 打印重合的股票代码
common_codes = common_df["代码"].unique()
print(f"\n重合的股票代码：{common_codes}")

# 如果没有重合
if common_df.empty:
    print("\n两个 DataFrame 之间没有重合的股票。")


In [ ]:
import akshare as ak

# 获取指数日线数据（示例：上证指数 sh000001）
symbol = "sh000001"
index_daily = ak.stock_zh_index_daily(symbol=symbol)

# 计算每日涨跌幅（收盘价相对于前一日的涨跌百分比）
index_daily['涨跌幅'] = index_daily['close'].pct_change() * 100

# 获取最新交易日尾盘（收盘）涨跌幅
latest_change = index_daily['涨跌幅'].iloc[-1]
print(f"最新交易日收盘涨跌幅: {latest_change:.2f}%")

In [ ]:
import akshare as ak
import pandas as pd
from datetime import datetime, timedelta

# 定义主要指数代码（格式：akshare 兼容的代码）
index_list = {
    "上证指数": "sh000001",
    "沪深300": "sh000300",
    "深证成指": "sz399001",
    "创业板指": "sz399006",
    "科创50": "sh000688"
}

def get_index_last_30min_change(symbol: str) -> float:
    """获取指数最后30分钟涨幅（基于分钟数据）"""
    try:
        # 获取分钟级数据（周期为1分钟）
        minute_data = ak.stock_board_concept_hist_min_em(symbol="上证指数", period="30")

        minute_data['日期时间'] = pd.to_datetime(minute_data['日期时间'])
        
        # 筛选交易日的最后30分钟（14:30-15:00）
        last_date = minute_data['日期时间'].max().date()  # 最新日期
        mask = (minute_data['日期时间'] >= pd.Timestamp(last_date).replace(hour=14, minute=30)) & \
               (minute_data['日期时间'] <= pd.Timestamp(last_date).replace(hour=15, minute=0))
        last_30min_data = minute_data[mask]
        
        if last_30min_data.empty:
            return 0.0
        
        # 计算涨幅：起始价（14:30的收盘价）到结束价（15:00的收盘价）
        start_price = last_30min_data.iloc[0]['收盘']
        end_price = last_30min_data.iloc[-1]['收盘']
        return (end_price - start_price) / start_price * 100
    except Exception as e:
        print(f"获取 {symbol} 分钟数据失败: {e}")
        return 0.0

# 获取各指数收盘价及最后30分钟涨幅
results = []
for index_name, symbol in index_list.items():
    # 获取日线数据（收盘价）
    daily_data = ak.stock_zh_index_daily(symbol=symbol)
    latest_close = daily_data['close'].iloc[-1]  # 最新收盘价
    
    # 获取最后30分钟涨幅
    last_30min_change = get_index_last_30min_change(symbol)
    
    results.append({
        "指数名称": index_name,
        "收盘价": round(latest_close, 2),
        "最后30分钟涨幅 (%)": round(last_30min_change, 2)
    })

# 转换为DataFrame输出
result_df = pd.DataFrame(results)
print("\nA股主要指数收盘价及尾盘涨幅:")
print(result_df)

In [ ]:
import akshare as ak

index_list = {
    "上证指数": "sh000001",
    "沪深300": "sh000300",
    "深证成指": "sz399001",
    "创业板指": "sz399006",
    "科创50": "sh000688"
}

for index_name, index_code in index_list.items():
    try:
        # 获取30分钟数据，period设置为'30'
        df = ak.stock_zh_index_minute(symbol=index_code, period='30')
        print(f"{index_name} ({index_code}) 的30分钟数据：")
        print(df.head())  # 打印前几行数据
    except Exception as e:
        print(f"获取 {index_name} ({index_code}) 数据失败，错误信息：{e}")

In [ ]:
import akshare as ak

# 定义指数列表
index_list = {
    "上证指数": "000001",
    "沪深300": "000300",
    "深证成指": "399001",
    "创业板指": "399006",
    "科创50": "000688"
}

# 遍历指数列表，获取数据
for index_name, index_code in index_list.items():
    try:
        # 使用 stock_zh_index_hist_min_em 方法获取指数数据
        df = ak.index_zh_a_hist(
            symbol=index_code, 
            period='30',  # 30分钟周期
            start_date="2025-02-26 09:30:00",  # 自定义开始日期
            end_date="2025-02-26 15:00:00"     # 自定义结束日期
        )
        print(f"{index_name} ({index_code}) 的30分钟数据：")
        print(df)
    except Exception as e:
        print(f"获取 {index_name} ({index_code}) 数据失败，错误信息：{e}")

In [ ]:
# 导入原始概念板块列表
from boards import concept_names

# 将概念板块分类
concept_categories = {
    "科技与数字化": {
        "AI与计算": [
            "人工智能", "云计算", "大数据", "算力概念", "边缘计算", "数字经济", "数字孪生", "东数西算",
            "ChatGPT概念", "AIGC概念", "AI芯片", "AI智能体", "AI语料", "AI制药", "AI手机", "AI眼镜",
            "智谱AI", "Kimi概念", "DeepSeek概念", "Sora概念", "多模态AI", "MLOps概念", "空间计算",
            "时空大数据", "商汤概念", "人脑工程", "AIPC", "机器视觉", "国资云概念",
        ],
        
        "芯片与半导体": [
            "半导体概念", "国产芯片", "存储芯片", "汽车芯片", "第三代半导体", "第四代半导体", 
            "IGBT概念", "碳化硅", "氮化镓", "中芯概念", "华为海思", "英伟达概念", "EDA概念", "Chiplet概念",
            "高带宽内存",
        ],
        
        "通信与网络": [
            "5G概念", "6G概念", "光通信模块", "F5G概念", "WiFi", "VPN", "天基互联", "量子通信",
            "毫米波概念", "UWB概念", "RCS概念", "eSIM", "物联网", "IPv6", "铜缆高速连接", 
            "星闪概念",
        ],
        
        "数字应用": [
            "区块链", "数字货币", "元宇宙概念", "NFT概念", "Web3.0", "虚拟数字人", "虚拟现实", "增强现实",
            "混合现实", "数据安全", "网络安全", "数据确权", "数据要素", "数字水印", "数字哨兵", "全息技术",
            "知识产权", "数据中心", "EDR概念", "超清视频"
        ],

        "硬件设备": [
            "LED", "OLED", "MiniLED", "MicroLED", "光刻机(胶)", "柔性屏(折叠屏)", 
            "玻璃基板", "PCB", "MLCC", "被动元件", "传感器", "激光雷达", "无线耳机",
            "智能穿戴", "电子车牌", "3D摄像头", "屏下摄像", "裸眼3D", "3D玻璃",
            "电子纸概念", "无线充电", "液冷概念", "SPD概念", 
        ],
        
        "互联网平台": [
            "华为概念", "华为欧拉", "华为昇腾", "鸿蒙概念", "阿里概念", "百度概念", 
            "腾讯云", "抖音概念(字节概念)", "快手概念", "拼多多概念", "小红书概念",
            "小米概念", "苹果概念", "富士康", "荣耀概念",
        ],
        
        "企业信息化": [
            "ERP概念", "智能家居", "智能电视", "智慧城市", "智慧政务", "智慧灯杆",
            "智能机器", "财税数字化", "DRG/DIP", "PLC概念", "信创", "国产软件",
            "电子身份证",
        ],

        "数字娱乐": [
            "网络游戏", "电子竞技", "云游戏", "短剧互动游戏", "手游概念"
        ]
    },
    
    "新能源与环保": {
        "清洁能源": [
            "新能源", "风能", "太阳能", "氢能源", "核能核电", "生物质能发电", 
            "地热能", "绿色电力", "可控核聚变",
        ],
        
        "能源设备": [
            "光伏设备", "光伏建筑一体化", "光伏高速公路", "特高压", "TOPCon电池",
            "超导概念", "虚拟电厂", "空气能热泵", "超超临界发电", "智能电网"
        ],
        
        "储能技术": [
            "储能", "熔盐储能", "抽水蓄能", "锂电池", "钠离子电池", "固态电池", 
            "钒电池", "燃料电池", "超级电容", "钙钛矿电池", "BC电池",
        ],

        "新能源汽车": [
            "新能源车", "动力电池回收", "麒麟电池", "刀片电池", "HIT电池",
        ],
        
        "节能减排": [
            "节能环保", "碳交易", "碳基材料", "低碳冶金", "尾气治理",
        ],
        
        "环境保护": [
            "垃圾分类", "土壤修复", "海绵城市", "核污染防治", "降解塑料",
        ],
    },
    
    "医药与健康": {
        "医疗器械": [
            "医疗器械概念", "体外诊断", "血氧仪", "注射器概念", "抗菌面料"
        ],
        
        "医疗服务": [
            "医疗服务", "医疗美容", "互联医疗", "CRO", "医废处理",
            "养老概念", "辅助生殖", "毛发医疗",
        ],
        
        "创新医药": [
            "创新药", "精准医疗", "基因测序", "单抗概念", "CAR-T细胞疗法",
            "免疫治疗", "生物识别", "合成生物", "重组蛋白", "纳米银",
        ],
        
        "传统医药": [
            "中药概念", "化学制药", "生物疫苗", "生物制品",
            "熊去氧胆酸", "地塞米松", "蒙脱石散", "青蒿素", 
            "千金藤素", "维生素", "肝素概念", "独家药品"
        ],

        "公共卫生": [
            "健康中国", "新冠药物", "疫苗冷链", "口罩", "消毒剂",
            "病毒防治", "流感", "气溶胶检测", "痘病毒防治",
            "超级真菌", "幽门螺杆菌概念", "阿兹海默", "肝炎概念",
        ],
        
        "健康管理": [
            "减肥药", "长寿药",
        ],
    },
    
    "消费与服务": {
        "食品饮料": [
            "白酒", "啤酒概念", "调味品概念", "食品安全", "乳业", "预制菜概念", 
            "代糖概念", "娃哈哈概念",
        ],
        
        "生活消费": [
            "化妆品概念", "美容护理", "宠物经济", "婴童概念", "托育服务", 
            "电子烟", "超级品牌", "谷子经济",
        ],
        
        "零售渠道": [
            "新零售", "电商概念", "跨境电商", "社区团购", "免税概念", 
            "抖音小店", "快递概念", "冷链物流", "进口博览", "万达概念", "内贸流通",
            "地摊经济",
        ],
        
        "服务业态": [
            "在线旅游", "在线教育", "网红直播", "盲盒经济", "职业教育",
            "共享经济", "远程办公", "彩票概念", "赛马概念",
        ],
        
        "商业模式": [
            "C2M概念", "新型工业化", "数字阅读",
        ],

        "文化体育": [
            "体育产业", "世界杯", "中超概念", "冰雪经济", "户外露营", "影视概念",
        ],
    },
    
    "制造与工业": {
        "先进制造": [
            "工业4.0", "工业互联", "智能制造", "专精特新", "工业母机", 
            "3D打印", "新型工业化",
        ],
        
        "机器人": [
            "机器人概念", "人形机器人", "机器人执行器", "减速器", "同步磁阻电机",
        ],
        
        "汽车工业": [
            "汽车整车", "汽车零部件", "汽车服务", "特斯拉", "华为汽车", "小米汽车", 
            "汽车热管理", "汽车一体化压铸", "无人驾驶", "车联网(车路云)", 
            "电子后视镜", "胎压监测", "轮毂电机", "充电桩", "换电概念", 
            "高压快充", "飞行汽车(eVTOL)", "汽车拆解",
        ],
        
        "工业材料": [
            "新材料", "稀土永磁", "石墨烯", "碳纤维", "稀缺资源", "工业大麻",
            "小金属概念", "锂矿概念", "有机硅", "PEEK材料概念", "培育钻石", "民爆概念",
        ],

        "工业设备": [
            "工程机械概念", "工业气体", "氦气概念", "装配建筑", "通用航空"
            "磁悬浮概念", "海工装备", "无人机", "复合集流体", "植物照明",
        ],
        
        "建筑工业": [
            "建筑节能", "铁路基建", "装配建筑", 
        ],
    },
    
    "金融与投资": {
        "金融服务": [
            "银行", "证券", "保险", "多元金融", "互联金融", "券商概念", "移动支付",
            "京东金融", "蚂蚁概念", "参股银行", "参股券商", "跨境支付", "ETC",
            "参股保险", "参股期货",
        ],
        
        "投资主题": [
            "融资融券", "创投", "REITs概念", "独角兽", "股权激励", "并购重组概念",
            "股权转让", "举牌", "基金重仓", "社保重仓", "机构重仓", "高送转",
            "QFII重仓", "证金持股", "转债标的", "贬值受益",
        ],

        "指数成分": [
            "沪股通", "深股通", "富时罗素", "MSCI中国", "标准普尔",
        ],

        "交易市场": [
            "B股", "AB股", "AH股", "GDR", "北交所概念", 
            "科创板做市商", "科创板做市股", "参股新三板", "IPO受益",
        ],
        
        "金融创新": [
            "纾困概念", "养老金", "租售同权", "首发经济", "化债(AMC)概念",
        ],
    },
    
    "国家战略与区域发展": {
        "区域发展战略": [
            "一带一路", "京津冀", "雄安新区", "长江三角", "粤港自贸", "东北振兴", "西部大开发",
            "成渝特区", "深圳特区", "上海自贸", "湖北自贸", "滨海新区", "乡村振兴", "新型城镇化",
            "土地流转", "沪企改革",
        ],
        
        "国企与军工": [
            "国企改革", "央企改革", "中字头", "军工", "航天航空", "航母概念", "大飞机", "军民融合",
            "国家安防", "商业航天", "北斗导航", "空间站概念",
        ],

        "国际合作": [
            "东盟自贸区概念", "RCEP概念", "中特估", "中俄贸易概念",
        ],
        
        "发展规划": [
            "2025规划", "统一大市场", "供销社概念", "PPP模式",
        ],
    },
    
    "资源与能源": {
        "传统能源": [
            "石油行业", "天然气", "煤炭行业", "煤化工", "页岩气", "油气设服", "可燃冰",
            "发电机概念",
        ],
        
        "农业资源": [
            "农业种植", "农牧饲渔", "水产养殖", "粮食概念", "生态农业", "转基因", "草甘膦",
            "猪肉概念", "鸡肉概念", "人造肉",
        ],
        
        "矿产资源": [
            "有色金属", "贵金属", "黄金概念", "钛白粉", "磷化工", "氟化工",
        ],

        "化工材料": [
            "化工原料", "环氧丙烷", "PVDF概念", "蓝宝石",
        ],
        
        "水资源": [
            "水利建设", "净水概念", "海洋经济",
        ],
    },
    

    "基建与公共服务": {
        "基础设施": [
            "地下管网", "低空经济", "广电",
        ],
        
        "公共服务": [
            "噪声防治", "房屋检测", "退税商店", "CPO概念",
        ],
    },

    "其他特色主题": {
        "特色指数": [
            "上证50_", "上证180_", "上证380", "中证500", "HS300_", "深证100R", "深成500",
            "创业成份", "创业板综", "央视50_", "茅指数", "宁组合",
        ],
        
        "特殊状态": [
            "ST股", "次新股", "注册制次新股", "低价股", "百元股", "微盘股", "破净股",
            "昨日涨停", "昨日涨停_含一字", "昨日连板", "昨日连板_含一字", "昨日触板",
            "预盈预增", "预亏预减", "壳资源",
        ],
    },
}

# 创建反向映射，用于检查分类情况
all_categorized = []
category_mapping = {}  # 记录每个概念属于哪个大类和子类
duplicates = []  # 记录重复分类的概念

# 遍历所有分类
for category, subcategories in concept_categories.items():
    if isinstance(subcategories, dict):
        # 处理有子类的大类
        for subcategory, concepts in subcategories.items():
            for concept in concepts:
                all_categorized.append(concept)
                # 检查是否重复分类
                if concept in category_mapping:
                    duplicates.append({
                        "concept": concept,
                        "original": category_mapping[concept],
                        "duplicate": {"category": category, "subcategory": subcategory}
                    })
                else:
                    category_mapping[concept] = {
                        "category": category,
                        "subcategory": subcategory
                    }
    else:
        # 处理没有子类的大类
        for concept in subcategories:
            all_categorized.append(concept)
            if concept in category_mapping:
                duplicates.append({
                    "concept": concept,
                    "original": category_mapping[concept],
                    "duplicate": {"category": category, "subcategory": None}
                })
            else:
                category_mapping[concept] = {
                    "category": category,
                    "subcategory": None
                }

# 检查未分类的概念
uncategorized = [concept for concept in concept_names if concept not in all_categorized]

# 打印分类结果
if __name__ == "__main__":
    # 打印分类统计
    print("\n=== 分类统计 ===")
    category_stats = {}
    for mapping in category_mapping.values():
        cat = mapping["category"]
        subcat = mapping["subcategory"]
        if cat not in category_stats:
            category_stats[cat] = {"total": 0, "subcategories": {}}
        category_stats[cat]["total"] += 1
        if subcat:
            if subcat not in category_stats[cat]["subcategories"]:
                category_stats[cat]["subcategories"][subcat] = 0
            category_stats[cat]["subcategories"][subcat] += 1

    for cat, stats in category_stats.items():
        print(f"\n{cat}（共{stats['total']}个概念）:")
        if stats["subcategories"]:
            for subcat, count in stats["subcategories"].items():
                print(f"  - {subcat}：{count}个")

    # 打印重复分类
    if duplicates:
        print("\n=== 重复分类 ===")
        for dup in duplicates:
            print(f"\n概念「{dup['concept']}」出现在多个分类中：")
            print(f"  - 原始分类：{dup['original']['category']}", end="")
            if dup['original']['subcategory']:
                print(f" / {dup['original']['subcategory']}")
            print(f"  - 重复分类：{dup['duplicate']['category']}", end="")
            if dup['duplicate']['subcategory']:
                print(f" / {dup['duplicate']['subcategory']}")

    # 打印未分类概念
    if uncategorized:
        print(f"\n=== 未分类概念（{len(uncategorized)}个）===")
        print(", ".join(uncategorized))

    # 打印总体统计
    print(f"\n=== 总体统计 ===")
    print(f"大类数量：{len(category_stats)}个")
    print(f"已分类概念：{len(category_mapping)}个")
    print(f"重复分类概念：{len(duplicates)}个")
    print(f"未分类概念：{len(uncategorized)}个")
    print(f"原始概念总数：{len(concept_names)}个")

In [ ]:
import adata
df = adata.stock.market.all_capital_flow_east(days_type=5)
filtered_df = df[df["index_code"] == "BK0800"]
print(filtered_df)

In [ ]:
import adata

res_dq = adata.stock.info.all_concept_code_east()
print(res_dq)

In [ ]:
res_dc = adata.stock.market.get_market_concept_east("BK1173")

print(res_dc)

In [ ]:
import adata

# 设置代理,代理是全局设置,代理失效后可重新设置。参数:ip,proxy_url
adata.proxy(is_proxy=False)

In [1]:

import adata
import json
import pandas as pd

# Get the capital flow data
df = adata.stock.market.all_capital_flow_east(days_type=5)

# Save to JSON file with proper encoding for Chinese characters using pandas

print(f"Data saved to capital_flow.json with {len(df)} records")

Data saved to capital_flow.json with 145 records


In [ ]:
#選股

#實時板塊數據
import adata
df = adata.stock.market.get_market_concept_current_east(index_code='BK0612')
print(df)


In [ ]:
import adata
df = adata.stock.market.get_capital_flow(stock_code='000001')
print(df)


In [36]:
def filter_high_potential_sectors(data):
    """
    基于多周期资金流数据的板块筛选函数
    新增功能：大额资金流三重维度风险分析
    """
    qualified_sectors = []
    
    for sector in data:
        # 获取周期数据；如果缺失，则跳过该板块
        d1 = sector.get('days_type_1')
        d5 = sector.get('days_type_5')
        d10 = sector.get('days_type_10')
        if not (d1 and d5 and d10):
            continue

        # 检查必备的字段是否存在，若缺失则跳过
        main_net_inflow_d10 = d10.get('main_net_inflow')
        main_net_inflow_d5 = d5.get('main_net_inflow')
        flow_1 = d1.get('main_net_inflow')
        if main_net_inflow_d10 is None or main_net_inflow_d5 is None or flow_1 is None:
            continue

        # 计算关键指标
        daily_10 = main_net_inflow_d10 / 10 if main_net_inflow_d10 else 0
        daily_5 = main_net_inflow_d5 / 5 if main_net_inflow_d5 else 0
        
        # 大额资金分析预处理（检查数据是否存在）
        lg_net_inflow_d10 = d10.get('lg_net_inflow')
        lg_net_inflow_d5 = d5.get('lg_net_inflow')
        lg_flow_1 = d1.get('lg_net_inflow')
        if lg_net_inflow_d10 is None or lg_net_inflow_d5 is None or lg_flow_1 is None:
            continue
        lg_daily_10 = lg_net_inflow_d10 / 10 if lg_net_inflow_d10 else 0
        lg_daily_5 = lg_net_inflow_d5 / 5 if lg_net_inflow_d5 else 0

        # 核心条件判断
        condition_continuity = (daily_10 <= daily_5 * 1.1) and (daily_5 <= flow_1 * 1.1)
        # 避免除零错误，若daily_10为0，则用1e-6代替
        strength_ratio = (flow_1 / (daily_10 + 1e-6)) * 100  
        condition_strength = strength_ratio > 150

        # 获取涨跌幅数据
        d1_change_pct = d1.get('change_pct')
        d5_change_pct = d5.get('change_pct')
        d10_change_pct = d10.get('change_pct')
        if d1_change_pct is None or d5_change_pct is None or d10_change_pct is None:
            continue

        # 量价健康度验证（注意：这里调整了顺序以匹配示例数据）
        price_condition = (d1_change_pct > 0 and d10_change_pct > d5_change_pct and d1_change_pct < 7)

        # 新增大额资金风险指标
        lg_strength_ratio = round((lg_flow_1 / (lg_daily_10 + 1e-6) * 100), 1)
        lg_acceleration = lg_daily_5 > lg_daily_10
        lg_price_divergence = (lg_flow_1 > 0) and (d1_change_pct < 1)
        lg_risk_indicators = {
            "lg_strength_ratio": lg_strength_ratio,
            "lg_acceleration": lg_acceleration,
            "lg_price_divergence": lg_price_divergence
        }

        # 主力资金验证（检查必要的流入率字段）
        d1_main_net_inflow_rate = d1.get('main_net_inflow_rate')
        d5_main_net_inflow_rate = d5.get('main_net_inflow_rate')
        d10_main_net_inflow_rate = d10.get('main_net_inflow_rate')
        if d1_main_net_inflow_rate is None or d5_main_net_inflow_rate is None or d10_main_net_inflow_rate is None:
            continue
        capital_condition = (d1_main_net_inflow_rate > 0.8 and
                             d5_main_net_inflow_rate > 0.3 and 
                             d10_main_net_inflow_rate > -0.1)
        
        if all([condition_continuity, condition_strength, price_condition, capital_condition]):
            sector_data = {
                "index_code": sector.get('index_code'),
                "index_name": sector.get('index_name'),
                "capital_flow": {
                    "strength_ratio": round(strength_ratio, 1),
                    "trend_type": "阶梯式流入" if strength_ratio < 300 else "短期过热",
                    "daily_sequence": [
                        {"days": 10, "net_inflow": d10.get('main_net_inflow'), "change_pct": d10.get('change_pct')},
                        {"days": 5, "net_inflow": d5.get('main_net_inflow'), "change_pct": d5.get('change_pct')},
                        {"days": 1, "net_inflow": d1.get('main_net_inflow'), "change_pct": d1.get('change_pct')}
                    ]
                },
                "risk_indicators": {
                    "small_capital_outflow": d1.get('sm_net_inflow', 0) < 0,
                    "mid_capital_behavior": "跟风" if d1.get('mid_net_inflow_rate', 0) > 0 else "撤离",
                    "large_capital_analysis": lg_risk_indicators
                }
            }
            qualified_sectors.append(sector_data)
    
    return qualified_sectors


# 假设您的原始数据格式如下
sample_data = [
    {
        "index_code": "BK1015",
        "index_name": "能源金属",
        "days_type_1": {
            "change_pct": 3.6,
            "main_net_inflow": 930234736.0,
            "main_net_inflow_rate": 5.61,
            "max_net_inflow": 529605232.0,
            "max_net_inflow_rate": 3.19,
            "lg_net_inflow": 400629504.0,
            "lg_net_inflow_rate": 2.42,
            "mid_net_inflow": -245510912.0,
            "mid_net_inflow_rate": -1.48,
            "sm_net_inflow": -684723760.0,
            "sm_net_inflow_rate": -4.13,
            "stock_code": "华友钴业",
            "stock_name": "603799"
        },
        "days_type_5": {
            "change_pct": 5.96,
            "main_net_inflow": 1550043536.0,
            "main_net_inflow_rate": 2.59,
            "max_net_inflow": 967757040.0,
            "max_net_inflow_rate": 1.62,
            "lg_net_inflow": 582286496.0,
            "lg_net_inflow_rate": 0.97,
            "mid_net_inflow": -31947936.0,
            "mid_net_inflow_rate": -0.05,
            "sm_net_inflow": -1518095536.0,
            "sm_net_inflow_rate": -2.54,
            "stock_code": "华友钴业",
            "stock_name": "603799"
        },
        "days_type_10": {
            "change_pct": 6.33,
            "main_net_inflow": 839000816.0,
            "main_net_inflow_rate": 0.93,
            "max_net_inflow": 513060016.0,
            "max_net_inflow_rate": 0.57,
            "lg_net_inflow": 325940800.0,
            "lg_net_inflow_rate": 0.36,
            "mid_net_inflow": 40740320.0,
            "mid_net_inflow_rate": 0.05,
            "sm_net_inflow": -879741072.0,
            "sm_net_inflow_rate": -0.97,
            "stock_code": "华友钴业",
            "stock_name": "603799"
        }
    },
]

# 执行筛选
result = filter_high_potential_sectors(sample_data)
print(result)


[{'index_code': 'BK1015', 'index_name': '能源金属', 'capital_flow': {'strength_ratio': 1108.7, 'trend_type': '短期过热', 'daily_sequence': [{'days': 10, 'net_inflow': 839000816.0, 'change_pct': 6.33}, {'days': 5, 'net_inflow': 1550043536.0, 'change_pct': 5.96}, {'days': 1, 'net_inflow': 930234736.0, 'change_pct': 3.6}]}, 'risk_indicators': {'small_capital_outflow': True, 'mid_capital_behavior': '撤离', 'large_capital_analysis': {'lg_strength_ratio': 1229.1, 'lg_acceleration': True, 'lg_price_divergence': False}}}]


In [25]:
import adata
import pandas as pd

# 获取股票资金流向数据
def get_stock_data(stock_code, start_date=None, end_date=None):
    df = adata.stock.market.get_capital_flow(stock_code=stock_code, start_date=start_date, end_date=end_date)
    return df

# 识别资金流向形态
def analyze_stock_fund_flow(df):
    df = df.sort_values(by='trade_date', ascending=True)  # 按日期升序排列
    df['net_flow_10d'] = df['lg_net_inflow'].rolling(window=10).sum()  # 10日资金净流入
    df['net_flow_5d'] = df['lg_net_inflow'].rolling(window=5).sum()  # 5日资金净流入
    df['net_flow_1d'] = df['lg_net_inflow']  # 1日资金净流入

    analysis_results = []

    for i in range(10, len(df)):  # 确保至少有10天的数据
        row = df.iloc[i]
        prev_row = df.iloc[i-1]

        # 形态1：阶梯式资金流入
        if (row['net_flow_10d'] > df.iloc[i-1]['net_flow_10d'] and
            row['net_flow_5d'] > df.iloc[i-1]['net_flow_5d'] and
            row['net_flow_1d'] > df.iloc[i-1]['net_flow_1d']):
            analysis_results.append((row['trade_date'], row['stock_code'], '阶梯式资金流入'))

        # 形态2：短期过热
        if (row['net_flow_1d'] > 3 * row['net_flow_5d'] / 5 and 
            row['net_flow_5d'] < df.iloc[i-1]['net_flow_5d']):
            analysis_results.append((row['trade_date'], row['stock_code'], '短期过热'))

        # 形态3：长期筑底反转
        if (row['net_flow_10d'] > df.iloc[i-1]['net_flow_10d'] and
            row['net_flow_5d'] > df.iloc[i-1]['net_flow_5d'] and
            row['net_flow_1d'] > 0 and
            prev_row['net_flow_1d'] < 0):  # 前一天资金流入为负，今日放量长阳
            analysis_results.append((row['trade_date'], row['stock_code'], '长期筑底反转'))

    return pd.DataFrame(analysis_results, columns=['trade_date', 'stock_code', 'pattern'])

# 示例：分析某只股票
stock_code = '000001'
df = get_stock_data(stock_code, "2025-03-04", "2025-02-18")
analysis_results = analyze_stock_fund_flow(df)

print(analysis_results)


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [ ]:
import adata
from tqdm import tqdm

def create_stock_concept_mapping():
    """
    创建股票-概念代码映射字典
    
    返回结构示例：
    {
        "600020": ["BK0685", "BK0700", "BK0683", ...],
        "000001": ["BK0521", "BK0732", ...],
        # 其他股票...
    }
    """
    try:
        # 获取全量股票并过滤北交所
        all_stock = adata.stock.info.all_code()
        filtered_stock = all_stock[all_stock['exchange'] != 'BJ']
        stock_codes = filtered_stock['stock_code'].unique().tolist()
    except Exception as e:
        print(f"获取股票列表失败：{str(e)}")
        return {}

    concept_map = {}
    
    with tqdm(stock_codes, desc='构建概念映射', unit='股') as pbar:
        for code in pbar:
            try:
                # 获取概念数据
                df = adata.stock.info.get_concept_east(stock_code=code)
                
                # 提取唯一概念代码
                if not df.empty:
                    concepts = df['concept_code'].drop_duplicates().tolist()
                else:
                    concepts = []
                
                concept_map[code] = concepts
                pbar.set_postfix({'当前股票': code, '概念数': len(concepts)})
                
            except Exception as e:
                print(f"\n股票 {code} 处理异常：{str(e)}")
                concept_map[code] = []
    
    return concept_map

# 使用示例
if __name__ == '__main__':
    mapping = create_stock_concept_mapping()
    
    # 验证样例输出
    sample_code = '600020'
    print(f"\n样例股票 {sample_code} 的概念代码：")
    print(mapping.get(sample_code, []))
    
    # 统计信息
    total_stocks = len(mapping)
    stocks_with_concepts = sum(1 for v in mapping.values() if v)

    print(f"\n统计：共{total_stocks}只股票，{stocks_with_concepts}只含概念，平均每只股票{avg_concepts:.1f}个概念")

构建概念映射: 100%|██████████| 5427/5427 [39:00<00:00,  2.32股/s, 当前股票=689009, 概念数=6] 


样例股票 600020 的概念代码：
['BK0980', 'BK0700', 'BK0683', 'BK0604', 'BK0506']


NameError: name 'total_stats' is not defined

In [39]:
total_stocks = len(mapping)
print(total_stocks)

5427


In [41]:
import json
from datetime import datetime
import os

def save_concept_map(concept_map, file_path=None):
    """
    将股票-概念映射字典保存为JSON文件
    
    参数:
    concept_map (dict): 通过create_stock_concept_mapping()生成的字典
    file_path (str): 可选的文件保存路径，默认当前目录
    
    返回:
    str: 最终保存的文件路径
    """
    try:
        # 生成带时间戳的文件名
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = f"stock_concept_map_{timestamp}.json"
        
        # 处理保存路径
        if file_path:
            os.makedirs(file_path, exist_ok=True)
            full_path = os.path.join(file_path, filename)
        else:
            full_path = filename
        
        # 写入文件
        with open(full_path, 'w', encoding='utf-8') as f:
            json.dump(concept_map, f, ensure_ascii=False, indent=2)
        
        # 验证文件大小
        file_size = os.path.getsize(full_path) / 1024  # 转换为KB
        print(f"保存成功！文件路径: {full_path}")
        print(f"文件大小: {file_size:.2f} KB")
        return full_path
    
    except Exception as e:
        print(f"保存失败: {str(e)}")
        return None

# 使用示例
if __name__ == '__main__':
    # 先创建映射字典
    concept_mapping = {
        "600020": ["BK0685", "BK0700", "BK0683", "BK0604", "BK0506"],
        "000001": ["BK0521", "BK0732", "BK0811"],
        "000002": ["BK0423", "BK0556", "BK0683"]
    }
    
    # 保存到当前目录
    saved_file = save_concept_map(mapping)
    
    # 保存到指定目录
    # save_concept_map(concept_mapping, file_path="data/stock_concepts")
    
    # 验证文件内容
    if saved_file and os.path.exists(saved_file):
        with open(saved_file, 'r', encoding='utf-8') as f:
            loaded_data = json.load(f)
            print("\n文件验证结果:")
            print(f"包含股票数量: {len(loaded_data)}")
            print(f"示例数据: {list(loaded_data.items())[:2]}")

保存成功！文件路径: stock_concept_map_20250305_155359.json
文件大小: 585.59 KB

文件验证结果:
包含股票数量: 5427
示例数据: [('000001', ['BK1071', 'BK0830', 'BK0637', 'BK0549']), ('000002', ['BK0992', 'BK0945', 'BK0822', 'BK0811', 'BK0882', 'BK0680', 'BK0653', 'BK0549'])]


In [42]:
import pytz
def get_today_date():
    """
    Returns today's date in 'YYYY-MM-DD' format using China Standard Time (CST).
    
    Returns:
        str: Today's date string in 'YYYY-MM-DD' format
    """
    # Get current time in China Standard Time
    china_tz = pytz.timezone('Asia/Shanghai')
    current_time_china = datetime.now(china_tz)
    
    # Format the date as YYYY-MM-DD
    formatted_date = current_time_china.strftime('%Y-%m-%d')
    
    return formatted_date

# Convert DataFrame to the desired JSON structure
def stock_capital_flow_df_to_json(dataframe):
    """
    Convert DataFrame to a specific JSON structure
    """
    if dataframe.empty:
        return {"stock_code": "", "capital_flow": []}
    
    # Get the stock code from the first row
    stock_code = dataframe['stock_code'].iloc[0]
    
    # Convert each row to a dictionary and build the data list
    data_list = []
    for _, row in dataframe.iterrows():
        data_item = {
            "trade_date": row['trade_date'],
            "main_net_inflow": float(row['main_net_inflow']),
            "sm_net_inflow": float(row['sm_net_inflow']),
            "mid_net_inflow": float(row['mid_net_inflow']),
            "lg_net_inflow": float(row['lg_net_inflow']),
            "max_net_inflow": float(row['max_net_inflow'])
        }
        data_list.append(data_item)
    
    # Create the final JSON structure
    result = {
        "stock_code": stock_code,
        "capital_flow": data_list
    }
    
    return result

In [48]:
import pandas as pd
import adata
from tqdm import tqdm
from datetime import datetime, timedelta

def get_recent_capital_flows(stock_list, days=10):
    """
    获取股票列表中各股票最近N个交易日的资金流向数据
    
    参数:
    stock_list (list): 股票代码列表，如 ['000001', '600000']
    days (int): 需要获取的交易日天数，默认为10
    
    返回:
    DataFrame: 包含所有股票资金流向的整合数据
    """
    # 初始化结果容器
    all_data = []
    
    # 创建进度条
    pbar = tqdm(stock_list, desc='获取资金流向', unit='股')
    
    for stock_code in pbar:
        try:
            # 获取原始数据（不指定日期范围）
            df = adata.stock.market.get_capital_flow(stock_code=stock_code, start_date = "2025-02-20")
            
            # 空数据检查
            if df.empty:
                pbar.set_postfix_str(f'{stock_code}: 无数据')
                continue
                
            # 数据预处理
            df = df.sort_values('trade_date', ascending=False)
            
            # 提取最近N个交易日
            recent_data = df.head(days).copy()
            
            # 添加处理标记
            recent_data['processed_time'] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            
            all_data.append(recent_data)
            pbar.set_postfix_str(f'{stock_code}: 成功获取{len(recent_data)}天数据')
            
        except Exception as e:
            print(f"\n股票 {stock_code} 处理失败：{str(e)}")
            continue
    
    # 合并数据
    if all_data:
        final_df = pd.concat(all_data, ignore_index=True)
        # 优化列顺序
        column_order = ['stock_code', 'trade_date', 'main_net_inflow',
                        'max_net_inflow', 'lg_net_inflow', 'mid_net_inflow',
                        'sm_net_inflow', 'processed_time']
        return final_df[column_order]
    else:
        return pd.DataFrame()

# 使用示例
if __name__ == '__main__':
    # 示例股票列表（实际可从文件或数据库读取）
    sample_stocks = ['000001', '600000', '000002', '600036', '000333']
    
    # 获取最近10个交易日数据
    capital_flows = get_recent_capital_flows(sample_stocks)
    
    # 结果展示
    if not capital_flows.empty:
        print("\n最近资金流向数据概览：")
        print(capital_flows.head())
        
        # 保存到CSV
        timestamp = datetime.now().strftime("%Y%m%d")
        filename = f"capital_flows_{timestamp}.csv"
        capital_flows.to_csv(filename, index=False, encoding='utf_8_sig')
        print(f"\n数据已保存至 {filename}")
    else:
        print("未获取到有效数据")

获取资金流向: 100%|██████████| 5/5 [00:04<00:00,  1.16股/s, 000333: 成功获取9天数据]


最近资金流向数据概览：
  stock_code  trade_date  main_net_inflow  max_net_inflow  lg_net_inflow  \
0     000001  2025-03-04       32952600.0      74209500.0    -41256900.0   
1     000001  2025-03-03       -4594200.0       -516000.0     -4078200.0   
2     000001  2025-02-28      -45980800.0     -27083700.0    -18897100.0   
3     000001  2025-02-27      121000000.0     144000000.0    -23582400.0   
4     000001  2025-02-26      -12659500.0     -27607300.0     14947800.0   

   mid_net_inflow  sm_net_inflow       processed_time  
0     -29555800.0     -3396800.0  2025-03-05 16:30:23  
1      15291700.0    -10697400.0  2025-03-05 16:30:23  
2      35873700.0     10107100.0  2025-03-05 16:30:23  
3     -47328700.0    -73494900.0  2025-03-05 16:30:23  
4      34001900.0    -21342400.0  2025-03-05 16:30:23  

数据已保存至 capital_flows_20250305.csv


In [46]:
import pandas as pd
import adata
from tqdm import tqdm
from datetime import datetime

def get_latest_min_capital_flows(stock_list):
    """
    获取各股票最新的分时资金流向数据（最后一个有效记录）
    
    参数:
    stock_list (list): 股票代码列表，如 ['000001', '600000']
    
    返回:
    DataFrame: 包含最新分时数据的整合结果
              列结构：[stock_code, trade_time, main_net_inflow, ..., processed_time]
    """
    results = []
    
    # 创建带状态显示的进度条
    with tqdm(stock_list, desc='采集分时数据', unit='股', colour='green') as pbar:
        for stock_code in pbar:
            try:
                # 获取原始分时数据
                df = adata.stock.market.get_capital_flow_min(stock_code=stock_code)
                
                # 空数据检查
                if df.empty:
                    pbar.set_postfix_str(f'{stock_code}: 无数据')
                    continue
                
                # 数据验证
                required_columns = ['stock_code', 'trade_time', 'main_net_inflow',
                                   'max_net_inflow', 'lg_net_inflow', 'mid_net_inflow',
                                   'sm_net_inflow']
                if not all(col in df.columns for col in required_columns):
                    pbar.set_postfix_str(f'{stock_code}: 数据字段缺失')
                    continue
                
                # 获取最新记录（假设数据按时间升序排列）
                latest_record = df.iloc[-1].copy()
                
                # 添加处理元数据
                latest_record['processed_time'] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
                
                results.append(latest_record)
                pbar.set_postfix_str(f'{stock_code}: 成功获取')
                
            except Exception as e:
                print(f"\n股票 {stock_code} 处理异常：{str(e)}")
                continue
    
    # 合并结果
    if results:
        final_df = pd.DataFrame(results)
        # 列顺序标准化
        column_order = required_columns + ['processed_time']
        return final_df[column_order]
    else:
        return pd.DataFrame()

# 使用示例
if __name__ == '__main__':
    # 测试用股票列表
    test_stocks = ['000001', '600000', '000002', '600036', '000333']
    
    # 执行数据采集
    min_flows = get_latest_min_capital_flows(test_stocks)
    
    # 结果处理
    if not min_flows.empty:
        print("\n最新分时资金流向数据：")
        print(min_flows.head())
        
        # 数据有效性检查
        print("\n数据统计：")
        print(f"总记录数：{len(min_flows)}")
        print(f"最新时间范围：{min_flows['trade_time'].min()} 至 {min_flows['trade_time'].max()}")
        
        # 保存到CSV
        timestamp = datetime.now().strftime("%Y%m%d_%H%M")
        filename = f"latest_min_capital_flows_{timestamp}.csv"
        min_flows.to_csv(filename, index=False, encoding='utf_8_sig')
        print(f"\n数据已保存至 {filename}")
    else:
        print("未获取到有效数据")

采集分时数据: 100%|██████████| 5/5 [00:09<00:00,  1.91s/股, 000333: 成功获取]


最新分时资金流向数据：
    stock_code          trade_time  main_net_inflow  max_net_inflow  \
239     000001 2025-03-05 15:00:00      196392730.0     182941028.0   
239     600000 2025-03-05 15:00:00       12463349.0     -11735561.0   
239     000002 2025-03-05 15:00:00      -97506560.0     -47637475.0   
239     600036 2025-03-05 15:00:00      239051729.0     236489475.0   
239     000333 2025-03-05 15:00:00      130989530.0     144331512.0   

     lg_net_inflow  mid_net_inflow  sm_net_inflow       processed_time  
239     13451702.0    -112132290.0    -84260411.0  2025-03-05 16:25:33  
239     24198910.0     -20720583.0      8257239.0  2025-03-05 16:25:35  
239    -49869085.0      19594639.0     77911927.0  2025-03-05 16:25:37  
239      2562254.0    -313063815.0     74012097.0  2025-03-05 16:25:39  
239    -13341982.0    -220586128.0     89596597.0  2025-03-05 16:25:41  

数据统计：
总记录数：5
最新时间范围：2025-03-05 15:00:00 至 2025-03-05 15:00:00

数据已保存至 latest_min_capital_flows_20250305_1625.csv
